![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx, and LangChain to make a series of calls to a language model

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook contains the steps and code to demonstrate Simple Sequential Chain using langchain integration with watsonx models.

Some familiarity with Python is helpful. This notebook uses Python 3.11.


## Learning goal

The goal of this notebook is to demonstrate how to chain `google/flan-ul2` and `google/flan-t5-xxl` models to generate a sequence of creating a random question on a given topic and an answer to that question and also to make the user friends with LangChain framework, using simple chain (LLMChain) and the extended chain (SimpleSequentialChain) with the WatsonxLLM.


## Contents

This notebook contains the following parts:

- [Setup](#setup)
- [Foundation Models on watsonx](#models)
- [WatsonxLLM interface](#watsonxllm)
- [Simple Sequential Chain experiment](#experiment)
- [Summary](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pack for Data administrator and ask him for your account credentials

### Install and import the `ibm-watsonx-ai` and dependecies
**Note:** `ibm-watsonx-ai` documentation can be found <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install "pydantic>=1.10.0" | tail -n 1
!pip install -U langchain | tail -n 1
!pip install -U langchain_ibm | tail -n 1

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud Pack for Data. You need to provide platform `url`, your `username` and `api_key`.

In [ ]:
username = 'PASTE YOUR USERNAME HERE'
api_key = 'PASTE YOUR API_KEY HERE'
url = 'PASTE THE PLATFORM URL HERE'

In [2]:
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    username=username,
    api_key=api_key,
    url=url,
    instance_id="openshift",
    version="5.0"
)

Alternatively you can use `username` and `password` to authenticate WML services.

```python
credentials = Credentials(
    username=***,
    password=***,
    url=***,
    instance_id="openshift",
    version="5.0"
)

```

### Defining the project id
The Foundation Model requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. Otherwise, please provide the project id.

In [3]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

<a id="models"></a>
## Foundation Models on `watsonx.ai`

#### List available models

All avaliable models are presented under `ModelTypes` class.

In [7]:
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes

print([model.name for model in ModelTypes])

['FLAN_T5_XXL', 'FLAN_UL2', 'MT0_XXL', 'GPT_NEOX', 'MPT_7B_INSTRUCT2', 'STARCODER', 'LLAMA_2_70B_CHAT', 'LLAMA_2_13B_CHAT', 'GRANITE_13B_INSTRUCT', 'GRANITE_13B_CHAT', 'FLAN_T5_XL', 'GRANITE_13B_CHAT_V2', 'GRANITE_13B_INSTRUCT_V2', 'ELYZA_JAPANESE_LLAMA_2_7B_INSTRUCT', 'MIXTRAL_8X7B_INSTRUCT_V01_Q', 'CODELLAMA_34B_INSTRUCT_HF', 'GRANITE_20B_MULTILINGUAL']


You need to specify `model_id`'s that will be used for inferencing:

In [8]:
model_id_1 = ModelTypes.FLAN_UL2
model_id_2 = ModelTypes.FLAN_T5_XXL

### Defining the model parameters

You might need to adjust model `parameters` for different models or tasks, to do so please refer to documentation under `GenTextParamsMetaNames` class.

**Action:** If any complications please refer to the <a href="https://ibm.github.io/watsonx-ai-python-sdk/fm_model.html#metanames.GenTextParamsMetaNames" target="_blank" rel="noopener no referrer">documentation</a>.

In [9]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.SAMPLE,
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.TEMPERATURE: 0.5,
    GenParams.TOP_K: 50,
    GenParams.TOP_P: 1
}

### Initialize the model
Initialize the `ModelInference` class with previous set params.

In [10]:
from ibm_watsonx_ai.foundation_models import ModelInference

flan_ul2_model = ModelInference(
    model_id=model_id_1, 
    params=parameters, 
    credentials=credentials,
    project_id=project_id)

flan_t5_model = ModelInference(
    model_id=model_id_2,
    credentials=credentials,
    project_id=project_id)

<a id="watsonxllm"></a>
## WatsonxLLM interface

`WatsonxLLM` is a wrapper around watsonx.ai models that provide chain integration around the models.

**Action:** For more details about `CustomLLM` check the [LangChain documentation](https://python.langchain.com/docs/modules/model_io/models/llms/custom_llm) 


### Initialize the `WatsonxLLM` class.

In [11]:
from langchain_ibm import WatsonxLLM

flan_ul2_llm = WatsonxLLM(watsonx_model=flan_ul2_model)
flan_t5_llm = WatsonxLLM(watsonx_model=flan_t5_model)

**Hint:** To use Chain interface from LangChain with watsonx.ai models you must call `model.to_langchain()` method. 

It returns `WatsonxLLM` wrapper compatible with LangChain CustomLLM specification.

In [ ]:
flan_ul2_model.to_langchain()

You can print all set data about the WatsonxLLM object using the `dict()` method.

In [13]:
flan_ul2_llm.dict()

{'model_id': 'google/flan-ul2',
 'deployment_id': None,
 'params': {'decoding_method': <DecodingMethods.SAMPLE: 'sample'>,
  'max_new_tokens': 100,
  'min_new_tokens': 1,
  'temperature': 0.5,
  'top_k': 50,
  'top_p': 1},
 'project_id': '4f35a756-b448-4f16-8237-c332aa79bc7a',
 'space_id': None,
 '_type': 'IBM watsonx.ai'}

<a id="experiment"></a>
## Simple Sequential Chain experiment
The simplest type of sequential chain is called a `SimpleSequentialChain`, in which each step has a single input and output and the output of one step serves as the input for the following step.

The experiment will consist in generating a random question about any topic and answer the following question.

An object called `PromptTemplate` assists in generating prompts using a combination of user input, additional non-static data, and a fixed template string.

In our case we would like to create two `PromptTemplate` objects which will be responsible for creating a random question and answering it.

In [14]:
from langchain import PromptTemplate

prompt_1 = PromptTemplate(
    input_variables=["topic"], 
    template="Generate a random question about {topic}: Question: "
)
prompt_2 = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question: {question}",
)

We would like to add functionality around language models using `LLMChain` chain. 

`prompt_to_flan_ul2` chain formats the prompt template whose task is to generate random question, passes the formatted string to LLM and returns the LLM output.

**Hint:** To use Chain interface from LangChain with watsonx.ai models you must call `model.to_langchain()` method. 

It returns `WatsonxLLM` wrapper compatible with LangChain CustomLLM specification.

In [15]:
from langchain.chains import LLMChain

prompt_to_flan_ul2 = LLMChain(llm=flan_ul2_model.to_langchain(), prompt=prompt_1)

`flan_to_t5` chain formats the prompt template whose task is to answer the question we got from `prompt_to_flan_ul2` chain, passes the formatted string to LLM and returns the LLM output.

In [16]:
flan_to_t5 = LLMChain(llm=flan_t5_model.to_langchain(), prompt=prompt_2)

This is the overall chain where we run `prompt_to_flan_ul2` and `flan_to_t5` chains in sequence.

In [17]:
from langchain.chains import SimpleSequentialChain

qa = SimpleSequentialChain(chains=[prompt_to_flan_ul2, flan_to_t5], verbose=True)

Generate random question and answer to topic.

In [19]:
qa.invoke('life')



> Entering new SimpleSequentialChain chain...
What is the smallest unit of life?
cell

> Finished chain.


{'input': 'life', 'output': 'cell'}

<a id="summary"></a>
## Summary and next steps

 You successfully completed this notebook!.
 
 You learned how to use Simple Squential Chain using custom llm WastonxLLM.
 
Check out our _<a href="https://ibm.github.io/watson-machine-learning-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors: 
 **Mateusz Szewczyk**, Software Engineer at Watson Machine Learning.

Copyright © 2023-2025 IBM. This notebook and its source code are released under the terms of the MIT License.